In [3]:
import sys
#print sys.executable
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import os

from nltk.corpus import stopwords
import time

from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
print('TensorFlow Version: {}' .format(tf.__version__))


TensorFlow Version: 1.1.0


In [2]:
data_dir = "/home/abhishekjindal/nlp/data/"
reviews = pd.read_csv(os.path.join(data_dir,"results-20180309-000248_8000_before_2010.csv"), encoding = "ISO-8859-1")

In [3]:
print(reviews.shape)
print(reviews.title[0])#.head(1).values.tolist()

(8000, 4)
Novel compounds for treatment of cancer and disorders associated with angiogenesis function


In [4]:
# Check for any nulls values
reviews.isnull().sum()

publication_date      0
publication_number    0
title                 0
abstract              0
dtype: int64

In [5]:
# Inspecting some of the reviews
for i in range(5):
    print("Patent #%d "%(i+1))
    print(reviews.title[i])
    print(reviews.abstract[i])
    print

Patent #1 
Novel compounds for treatment of cancer and disorders associated with angiogenesis function
Novel compounds for treatment of cancer and disorders associated with angiogenesis function. Also disclosed are a method of preparing the compounds, pharmaceutical compositions and packaged products containing the compounds, a method of using these molecules to treat cancer (e.g., leukemia, non-small cell lung cancer, colon cancer, CNS cancer, melanoma, ovarian cancer, breast cancer, renal cancer, and prostate cancer) and disorders associated with angiogenesis function (e.g., age-related macular degeneration, macular dystrophy, and diabetes), a method of monitoring the treatment, a method of profiling gene expression, and a method of modulating cell growth, cell cycle, apoptosis, or gene expression.
Patent #2 
Substituted 2-thio-3,5-dicyano-4-aryl-6-aminopyridines and their use
The invention relates to compounds of general formula (I), a method for the production thereof and the use t

In [6]:
# A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [7]:
def clean_text(text, remove_stopwords = True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''
    
    # Convert words to lower case
    text = text.lower()
    
    # Replace contractions with their longer forms 
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)
    
    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text) 
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [8]:
# Clean the summaries and texts
clean_summaries = []
for title in reviews.title:
    clean_summaries.append(clean_text(title, remove_stopwords=False))
print("Texts are complete.")


clean_texts = []

for abstract in reviews.abstract:
    clean_texts.append(clean_text(abstract))
print("Summaries are complete.")

Texts are complete.
Summaries are complete.


In [9]:
for i in range(5):
    print("Patent #%d "%(i+1))
    print(clean_summaries[i])
    print(clean_texts[i])
    print

Patent #1 
novel compounds for treatment of cancer and disorders associated with angiogenesis function
novel compounds treatment cancer disorders associated angiogenesis function also disclosed method preparing compounds pharmaceutical compositions packaged products containing compounds method using molecules treat cancer e g leukemia non small cell lung cancer colon cancer cns cancer melanoma ovarian cancer breast cancer renal cancer prostate cancer disorders associated angiogenesis function e g age related macular degeneration macular dystrophy diabetes method monitoring treatment method profiling gene expression method modulating cell growth cell cycle apoptosis gene expression
Patent #2 
substituted 2 thio 3 5 dicyano 4 aryl 6 aminopyridines and their use
invention relates compounds general formula method production thereof use thereof pharmacologically effective substances broad medical indication spectrum furthermore selective adenosine receptor ligands preferably selective adeno

In [10]:
def count_words(count_dict, text):
    '''Count the number of occurrences of each word in a set of text'''
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [11]:
# Find the number of times each word was used and the size of the vocabulary
word_counts = {}

count_words(word_counts, clean_summaries)
count_words(word_counts, clean_texts)
            
print("Size of Vocabulary:", len(word_counts))

Size of Vocabulary: 23187


In [4]:
embeddings_index = {}
with open('/home/abhishekjindal/nlp/data/glove.840B.300d.txt') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 2196016


In [13]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold = 0

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round((1.0*missing_words/len(word_counts))*100,4)
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 2419
Percent of words that are missing from vocabulary: 10.4326%


In [17]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in GloVe

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(1.0*len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 23187
Number of words we will use: 20797
Percent of words we will use: 89.69%


In [18]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

20797


In [19]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts'''
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
    return ints, word_count, unk_count

In [20]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_summaries, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count)
int_texts, word_count, unk_count = convert_to_ints(clean_texts, word_count, unk_count, eos=True)

unk_percent = round(1.0*unk_count/word_count,4)*100

print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 610988
Total number of UNKs in headlines: 3888
Percent of words that are UNK: 0.64%


In [21]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [22]:
lengths_summaries = create_lengths(int_summaries)
lengths_texts = create_lengths(int_texts)

print("Summaries:")
print(lengths_summaries.describe())
print()
print("Texts:")
print(lengths_texts.describe())

Summaries:
            counts
count  8000.000000
mean      8.096875
std       4.597949
min       1.000000
25%       5.000000
50%       7.000000
75%      11.000000
max      48.000000

Texts:
            counts
count  8000.000000
mean     69.276625
std      29.684301
min       5.000000
25%      48.000000
50%      67.000000
75%      86.000000
max     389.000000


In [23]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))

105.0
122.0
154.0


In [24]:
# Inspect the length of summaries
print(np.percentile(lengths_summaries.counts, 90))
print(np.percentile(lengths_summaries.counts, 95))
print(np.percentile(lengths_summaries.counts, 99))

14.0
17.0
23.0


In [25]:
def unk_counter(sentence):
    '''Counts the number of time UNK appears in a sentence.'''
    unk_count = 0
    for word in sentence:
        if word == vocab_to_int["<UNK>"]:
            unk_count += 1
    return unk_count

In [26]:
# Sort the summaries and texts by the length of the texts, shortest to longest
# Limit the length of summaries and texts based on the min and max ranges.
# Remove reviews that include too many UNKs

sorted_summaries = []
sorted_texts = []
max_text_length = 1000
max_summary_length = 100
min_length = 2
unk_text_limit = 7
unk_summary_limit = 3

for length in range(min(lengths_texts.counts), max_text_length): 
    for count, words in enumerate(int_summaries):
        if (len(int_summaries[count]) >= min_length and
            len(int_summaries[count]) <= max_summary_length and
            len(int_texts[count]) >= min_length and
            unk_counter(int_summaries[count]) <= unk_summary_limit and
            unk_counter(int_texts[count]) <= unk_text_limit and
            length == len(int_texts[count])
           ):
            sorted_summaries.append(int_summaries[count])
            sorted_texts.append(int_texts[count])
        
# Compare lengths to ensure they match
print(len(sorted_summaries))
print(len(sorted_texts))

7675
7675


In [27]:
def model_inputs():
    '''Create palceholders for inputs to the model'''
    
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    summary_length = tf.placeholder(tf.int32, (None,), name='summary_length')
    max_summary_length = tf.reduce_max(summary_length, name='max_dec_len')
    text_length = tf.placeholder(tf.int32, (None,), name='text_length')

    return input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length

In [28]:
def process_encoding_input(target_data, vocab_to_int, batch_size):
    '''Remove the last word id from each batch and concat the <GO> to the begining of each batch'''
    
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [29]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob):
    '''Create the encoding layer'''
    
    for layer in range(num_layers):
        with tf.variable_scope('encoder_{}'.format(layer)):
            cell_fw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                    input_keep_prob = keep_prob)

            cell_bw = tf.contrib.rnn.LSTMCell(rnn_size,
                                              initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                    input_keep_prob = keep_prob)

            enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                    cell_bw, 
                                                                    rnn_inputs,
                                                                    sequence_length,
                                                                    dtype=tf.float32)
    # Join outputs since we are using a bidirectional RNN
    enc_output = tf.concat(enc_output,2)
    
    return enc_output, enc_state

In [30]:
def training_decoding_layer(dec_embed_input, summary_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_summary_length):
    '''Create the training logits'''
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=summary_length,
                                                        time_major=False)

    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       initial_state,
                                                       output_layer) 

    training_logits, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                           output_time_major=False,
                                                           impute_finished=True,
                                                           maximum_iterations=max_summary_length)
    return training_logits

In [31]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_summary_length, batch_size):
    '''Create the inference logits'''
    
    start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                start_tokens,
                                                                end_token)
                
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        initial_state,
                                                        output_layer)
                
    inference_logits, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            output_time_major=False,
                                                            impute_finished=True,
                                                            maximum_iterations=max_summary_length)
    
    return inference_logits

In [32]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, text_length, summary_length, 
                   max_summary_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers):
    '''Create the decoding cell and attention for the training and inference decoding layers'''
    
    for layer in range(num_layers):
        with tf.variable_scope('decoder_{}'.format(layer)):
            lstm = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
            dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                     input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  text_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')

    dec_cell = tf.contrib.seq2seq.DynamicAttentionWrapper(dec_cell,
                                                          attn_mech,
                                                          rnn_size)
            
    initial_state = tf.contrib.seq2seq.DynamicAttentionWrapperState(enc_state[0],
                                                                    _zero_state_tensors(rnn_size, 
                                                                                        batch_size, 
                                                                                        tf.float32)) 
    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input, 
                                                  summary_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_summary_length)
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_summary_length,
                                                    batch_size)

    return training_logits, inference_logits

In [33]:
def seq2seq_model(input_data, target_data, keep_prob, text_length, summary_length, max_summary_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size):
    '''Use the previous functions to create the training and inference logits'''
    
    # Use Numberbatch's embeddings and the newly created ones as our embeddings
    embeddings = word_embedding_matrix
    
    enc_embed_input = tf.nn.embedding_lookup(embeddings, input_data)
    enc_output, enc_state = encoding_layer(rnn_size, text_length, num_layers, enc_embed_input, keep_prob)
    
    dec_input = process_encoding_input(target_data, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        text_length, 
                                                        summary_length, 
                                                        max_summary_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers)
    
    return training_logits, inference_logits

In [34]:
def pad_sentence_batch(sentence_batch):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [35]:
def get_batches(summaries, texts, batch_size):
    """Batch summaries, texts, and the lengths of their sentences together"""
    for batch_i in range(0, len(texts)//batch_size):
        start_i = batch_i * batch_size
        summaries_batch = summaries[start_i:start_i + batch_size]
        texts_batch = texts[start_i:start_i + batch_size]
        pad_summaries_batch = np.array(pad_sentence_batch(summaries_batch))
        pad_texts_batch = np.array(pad_sentence_batch(texts_batch))
        
        # Need the lengths for the _lengths parameters
        pad_summaries_lengths = []
        for summary in pad_summaries_batch:
            pad_summaries_lengths.append(len(summary))
        
        pad_texts_lengths = []
        for text in pad_texts_batch:
            pad_texts_lengths.append(len(text))
        
        yield pad_summaries_batch, pad_texts_batch, pad_summaries_lengths, pad_texts_lengths

In [36]:
# Set the Hyperparameters
epochs = 15
batch_size = 64
rnn_size = 256
num_layers = 2
learning_rate = 0.005
keep_probability = 0.75

In [37]:
# Build the graph
train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, keep_prob, summary_length, max_summary_length, text_length = model_inputs()

    # Create the training and inference logits
    training_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      text_length,
                                                      summary_length,
                                                      max_summary_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size)
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_logits.rnn_output, 'logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(summary_length, max_summary_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(learning_rate)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)
print("Graph is built.")

Graph is built.


In [38]:
# Subset the data for training
start = 0#200000
end = start + 5000
sorted_summaries_short = sorted_summaries[start:end]
sorted_texts_short = sorted_texts[start:end]
print(len(sorted_summaries))
print("The shortest text length:", len(sorted_texts_short[0]))
print("The longest text length:",len(sorted_texts_short[-1]))

7675
The shortest text length: 5
The longest text length: 78


In [39]:
# Train the Model
learning_rate_decay = 0.95
min_learning_rate = 0.0005
display_step = 20 # Check training loss after every 20 batches
stop_early = 0 
stop = 3 # If the update loss does not decrease in 3 consecutive update checks, stop training
per_epoch = 3 # Make 3 update checks per epoch
update_check = (len(sorted_texts_short)//batch_size//per_epoch)-1
print(update_check)
update_loss = 0 
batch_loss = 0
summary_update_loss = [] # Record the update losses for saving improvements in the model

checkpoint = "best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
    
    # If we want to continue training a previous session
    #loader = tf.train.import_meta_graph("./" + checkpoint + '.meta')
    #loader.restore(sess, checkpoint)
    
    for epoch_i in range(1, epochs+1):
        update_loss = 0
        batch_loss = 0
        for batch_i, (summaries_batch, texts_batch, summaries_lengths, texts_lengths) in enumerate(
                get_batches(sorted_summaries_short, sorted_texts_short, batch_size)):
            start_time = time.time()
            _, loss = sess.run(
                [train_op, cost],
                {input_data: texts_batch,
                 targets: summaries_batch,
                 lr: learning_rate,
                 summary_length: summaries_lengths,
                 text_length: texts_lengths,
                 keep_prob: keep_probability})

            batch_loss += loss
            update_loss += loss
            end_time = time.time()
            batch_time = end_time - start_time

            if batch_i % display_step == 0 and batch_i > 0:
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(sorted_texts_short) // batch_size, 
                              batch_loss / display_step, 
                              batch_time*display_step))
                batch_loss = 0

            if batch_i % update_check == 0 and batch_i > 0:
                print("Average loss for this update:", round(update_loss/update_check,3))
                summary_update_loss.append(update_loss)
                
                # If the update loss is at a new minimum, save the model
                if update_loss <= min(summary_update_loss):
                    print('New Record!') 
                    stop_early = 0
                    saver = tf.train.Saver() 
                    saver.save(sess, checkpoint)

                else:
                    print("No Improvement.")
                    stop_early += 1
                    if stop_early == stop:
                        break
                update_loss = 0
            
                    
        # Reduce learning rate, but not below its minimum value
        learning_rate *= learning_rate_decay
        if learning_rate < min_learning_rate:
            learning_rate = min_learning_rate
        
        if stop_early == stop:
            print("Stopping Training.")
            break

25
Epoch   1/15 Batch   20/78 - Loss:  3.934, Seconds: 45.06
Average loss for this update: 3.742
New Record!
Epoch   1/15 Batch   40/78 - Loss:  2.860, Seconds: 48.35
Average loss for this update: 2.683
New Record!
Epoch   1/15 Batch   60/78 - Loss:  2.515, Seconds: 81.61
Average loss for this update: 2.378
New Record!
Epoch   2/15 Batch   20/78 - Loss:  2.521, Seconds: 43.84
Average loss for this update: 2.494
No Improvement.
Epoch   2/15 Batch   40/78 - Loss:  2.312, Seconds: 49.02
Average loss for this update: 2.199
New Record!
Epoch   2/15 Batch   60/78 - Loss:  2.097, Seconds: 81.70
Average loss for this update: 1.979
New Record!
Epoch   3/15 Batch   20/78 - Loss:  2.173, Seconds: 44.20
Average loss for this update: 2.145
No Improvement.
Epoch   3/15 Batch   40/78 - Loss:  1.971, Seconds: 48.68
Average loss for this update: 1.889
New Record!
Epoch   3/15 Batch   60/78 - Loss:  1.813, Seconds: 81.48
Average loss for this update: 1.698
New Record!
Epoch   4/15 Batch   20/78 - Loss: 

In [40]:
def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [41]:
# Create your own review or use one from the dataset
#input_sentence = "I have never eaten an apple before, but this red one was nice. \
                  #I think that I will try a green apple next time."
#text = text_to_seq(input_sentence)
title = "novel compounds for treatment of cancer and disorders associated with angiogenesis function"
 
input_sentence= "novel compounds treatment cancer disorders associated angiogenesis function also disclosed method preparing compounds pharmaceutical compositions packaged products containing compounds method using molecules treat cancer e g leukemia non small cell lung cancer colon cancer cns cancer melanoma ovarian cancer breast cancer renal cancer prostate cancer disorders associated angiogenesis function e g age related macular degeneration macular dystrophy diabetes method monitoring treatment method profiling gene expression method modulating cell growth cell cycle apoptosis gene expression"
text = text_to_seq(input_sentence)
#random = np.random.randint(0,len(clean_texts))
#input_sentence = clean_texts[random]
#title = clean_summaries[random]
#text = text_to_seq(clean_texts[random])

checkpoint = "./best_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(10,20)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 

# Remove the padding from the tweet
pad = vocab_to_int["<PAD>"] 

print('Original Title:', title)

print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from ./best_model.ckpt
Original Title: novel compounds for treatment of cancer and disorders associated with angiogenesis function

Text
  Word Ids:    [9416, 1730, 11926, 18340, 1766, 1440, 9340, 9851, 8476, 12789, 13931, 14052, 1730, 11389, 15281, 18011, 18353, 7576, 1730, 13931, 3896, 12766, 8942, 18340, 17049, 12192, 1933, 14459, 16939, 9985, 12017, 18340, 2362, 18340, 16866, 18340, 107, 10758, 18340, 11837, 18340, 1902, 18340, 8593, 18340, 1766, 1440, 9340, 9851, 17049, 12192, 345, 3843, 7445, 12201, 7445, 12917, 233, 13931, 19655, 11926, 13931, 15993, 14186, 16656, 13931, 7277, 9985, 1907, 9985, 12063, 20278, 14186, 16656]
  Input Words: novel compounds treatment cancer disorders associated angiogenesis function also disclosed method preparing compounds pharmaceutical compositions packaged products containing compounds method using molecules treat cancer e g leukemia non small cell lung cancer colon cancer cns cancer melanoma ovarian cancer br

In [43]:
import json
from rouge import Rouge

hyps = "this is a test"
refs = "a test is this"
rouge = Rouge()
scores = rouge.get_scores(hyps, refs)

print(scores)

[{'rouge-2': {'p': 0.3333333333333333, 'r': 0.3333333333333333, 'f': 0.3333333283333334}, 'rouge-l': {'p': 0.5, 'r': 0.5, 'f': 0.4999999999995}, 'rouge-1': {'p': 1.0, 'r': 1.0, 'f': 0.999999995}}]


In [44]:
clean_text_top10 = clean_texts[0:100]
clean_summaries_top10 = clean_summaries[0:100]

In [45]:
output_summaries = []

In [46]:
output_summaries = []
checkpoint = "./best_model.ckpt"
pad = vocab_to_int["<PAD>"] 
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    #Multiply by batch_size to match the model's input parameters
    for i in range(0, len(clean_summaries_top10)):
        text = text_to_seq(clean_text_top10[i])
        answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(10,20)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 
        temp = " ".join([int_to_vocab[i] for i in answer_logits if i != pad])
        output_summaries.append(temp)
        if(i % 50 == 0):
            print("Current iter: ", i)

INFO:tensorflow:Restoring parameters from ./best_model.ckpt
Current iter:  0
Current iter:  50


In [47]:
scores = rouge.get_scores(output_summaries, clean_summaries_top10, avg = True)
print(scores)

{'rouge-2': {'p': 0.4371833999333999, 'r': 0.4078098788466436, 'f': 0.40216918712049965}, 'rouge-l': {'p': 0.5493473565907777, 'r': 0.5216811997514009, 'f': 0.48444530245081224}, 'rouge-1': {'p': 0.6522189199689198, 'r': 0.5471301929065087, 'f': 0.5742984883804074}}


In [52]:
index = 59
print(output_summaries[index])
print(clean_summaries_top10[index])

adjustable tray assembly
slat assembly for blind


In [53]:
print(clean_text_top10[index])

slat assembly blind includes plurality column like support rods board like support blocks alternatively arranged juxtaposition others adhesive layer coated bottom surface thereon woven fabric various diagrams attached thereto via adhesive layer thereof weaving spots woven fabric adjacent joints juxtaposed support rods blocks thereof securely bound together form rigid straight piece decoration article cut equal space plurality slat pieces cord passage hole disposed lateral sides thereon respectively form horizontal type slat piece hook hole disposed one lateral side thereon provide vertical type slat piece thereof thus slat pieces turned rolled alternatively arranged support rods blocks woven fabric thereof verify diagrams displayed thereon achieve special visual effects blind assembly thereof besides via weaving texture woven fabric slat pieces thereof equipped rich colors three dimensional diagrams well via adhesive layer woven fabric securely bound support rods blocks efficiently avo

In [54]:
data_dir_test = "/home/abhishekjindal/nlp/data/"
reviews_test = pd.read_csv(os.path.join(data_dir,"results-20180309-000423_8000_after_2010.csv"), encoding = "ISO-8859-1")

In [55]:
# Clean the Test summaries and texts
clean_summaries_test = []
for title in reviews_test.title:
    clean_summaries_test.append(clean_text(title, remove_stopwords=False))
print("Texts are complete.")


clean_texts_test = []

for abstract in reviews_test.abstract:
    clean_texts_test.append(clean_text(abstract))
print("Summaries are complete.")

Texts are complete.
Summaries are complete.


In [58]:
clean_text_test_sample = clean_texts_test[0:500]
clean_summaries_test_sample = clean_summaries_test[0:500]

In [60]:
output_summaries_test = []
checkpoint = "./best_model.ckpt"
pad = vocab_to_int["<PAD>"] 
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    
    #Multiply by batch_size to match the model's input parameters
    for i in range(0, len(clean_text_test_sample)):
        text = text_to_seq(clean_text_test_sample[i])
        answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      summary_length: [np.random.randint(10,20)], 
                                      text_length: [len(text)]*batch_size,
                                      keep_prob: 1.0})[0] 
        temp = " ".join([int_to_vocab[i] for i in answer_logits if i != pad])
        output_summaries_test.append(temp)
        if(i % 50 == 0):
            print("Current iter: ", i)

INFO:tensorflow:Restoring parameters from ./best_model.ckpt
Current iter:  0
Current iter:  50
Current iter:  100
Current iter:  150
Current iter:  200
Current iter:  250
Current iter:  300
Current iter:  350
Current iter:  400
Current iter:  450


In [61]:
scores = rouge.get_scores(output_summaries_test, clean_summaries_test_sample, avg = True)
print(scores)

{'rouge-2': {'p': 0.10147532745032746, 'r': 0.07822435668906257, 'f': 0.07862278668023955}, 'rouge-l': {'p': 0.24077211448682034, 'r': 0.19748969917447087, 'f': 0.1774380853254873}, 'rouge-1': {'p': 0.2965377733377733, 'r': 0.21940039571711398, 'f': 0.23506253605943586}}


In [62]:
import nltk

hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room']
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room']
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)

0.4548019047027907


In [ ]:
clean_summaries_test_sample_v1 = []
for i in range(0, len(clean_summaries_test_sample)):
    clean_summaries_test_sample_v1.append([clean_summaries_test_sample[i]])

In [85]:
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
cc = SmoothingFunction()
BLEUscore = nltk.translate.bleu_score.sentence_bleu(clean_summaries_test_sample_v1, output_summaries_test, smoothing_function=cc.method4)
print(BLEUscore)

0.16420916716310238


In [91]:
index = 15
print(output_summaries_test[index])
print(clean_summaries_test_sample[index])
print(clean_text_test_sample[index])

liquid crystal display panel panel panel panel substrate
liquid crystal display panel and manufacturing method thereof
present disclosure provides liquid crystal display panel liquid crystal display panel includes first substrate second substrate liquid crystal layer liquid crystal layer first substrate second substrate first substrate includes first alignment layer second substrate includes second alignment layer first alignment layer photo alignment layer second substrate rubbing alignment layer


In [87]:
import nltk

hypothesis = ['It', 'is', 'a', 'cat', 'at', 'room']
reference = ['It', 'is', 'a', 'cat', 'inside', 'the', 'room']
#there may be several references
BLEUscore = nltk.translate.bleu_score.sentence_bleu([reference], hypothesis)
print(BLEUscore)

0.4548019047027907


In [88]:
clean_summaries_top10_v1 = []
for i in range(0, len(clean_summaries_top10)):
    clean_summaries_top10_v1.append([clean_summaries_top10[i]])

In [89]:
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
cc = SmoothingFunction()
BLEUscore = nltk.translate.bleu_score.sentence_bleu(clean_summaries_top10_v1, output_summaries, smoothing_function=cc.method4)
print(BLEUscore)

0.28095777279645495


In [82]:
BLEUscore = nltk.translate.bleu_score.sentence_bleu(clean_summaries_test_sample_v1[3], output_summaries_test[3])
print(BLEUscore)

0.27160842897599496
